# 第3部 第6章 ダミー変数と分散分析モデル

## はじめに

## フォルダ構造とユーティリティ関数、ライブラリimport
リンク集の記事にフォルダ構造とユーティリティ関数、ライブラリimportを掲載しました。\
準備としてそちらのページをご覧ください。
1. [フォルダ構造とユーティリティ関数]()
1. [ライブラリimport]()

## モジュールのimport

In [ ]:
# Module
import sys
sys.path.append("../")
from mod.numpyro_utility import *

# DataFrame, Numerical computation
import polars as pl
pl.Config(fmt_str_lengths = 100, tbl_cols = 100, tbl_rows = 100)
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

# ベイズ推定
import numpyro
import numpyro.distributions as dist # 確率分布

# plot
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import arviz as az

# plotの設定
import json
def to_rc_dict(dict):
    """
    jsonファイルのdictを読み込む
    """
    return {f'{k1}.{k2}': v for k1,d in dict.items() for k2,v in d.items()}

file_path = "../mod/rcParams.json"
with open(file_path) as f: 
    plt.rcParams.update(to_rc_dict(json.load(f)))

# 日本語 or 英語の2択
import japanize_matplotlib
#plt.rcParams['font.family'] = "Times New Roman"

## 6.1 本章の目的と概要
参考書籍から引用します。\
RとStanやbrmsの部分をPythonとNumPyroに読み替えてください。

> **テーマ**\
> 本章ではダミー変数について学び,これを用いた分散分析モデルを brms や rstan で推定します。

> **目的**\
> 説明変数として質的データを用いたモデリングの方法を学んでいただくために,本性を執筆しました.
> 分散分析モデルを対象としますが,他の様々なﾓﾃﾞﾙでも,ダミー変数を活用することは可能です.


> **概要**\
> モデルの構造 → 分析の準備 → データの読み込みと可視化 → brms による分散分析モデルの推定\
> → (補足)分散分析モデルのデザイン行列 → (補足)brmsを使わない分散分析モデルの推定

## 6.2 モデルの構造
ビールの売り上げ $y_{i}$ と天候 $( x_{i, 1}, x_{i, 2} )$ との関係を数式で整理します。\
参考書籍では次のように書かれています。\
ここで $x_{i,1}$ は晴れの日に $1$ , そうでない場合に $0$ を取ります。\
$x_{i,2}$ は雨の日に $1$ , そうでない場合に $0$ を取ります。\
$( x_{i, 1}, x_{i, 2} ) = (0, 0)$ の場合は曇りを表します。

$$
\begin{aligned}
\mu_{i} =& \beta_{0} + \beta_{1} x_{i,1} + \beta_{2} x_{i,2}\\
y_{i} \sim& Normal(\mu_{i}, \sigma^2)
\end{aligned}
$$

回帰分析と同様に確率モデルのパラメータの事前分布からモデル化します。

## 6.3 分析の準備
ライブラリの準備をしている節です。\
省略します。

## 6.4 データの読み込みと可視化
データを読み込んで概要を把握します。

In [ ]:
# データを読み込む
df = pl.read_csv("../data/3-6-1-beer-sales-3.csv")
display(df.head())

# サンプルサイズを表示する
print( len(df) )

天候と売り上げの関係をバイオリンプロットで可視化します。\
晴れの日の売り上げが高いことがわかります。

In [ ]:
ax = sns.violinplot(data = df, x = "weather", y = "sales", hue = "weather", inner = "point")
ax.set_title("図3.6.1 ビールの売り上げと天気のバイオリンプロット")

## 6.5 brms による分散の推定
Pythonにおけるbrms相当のライブラリにbambiというライブラリがあります。\
私にとって今はまだこれを使う時ではないので、省略します。

## 6.6 補足:分散分析モデルのデザイン行列
前節と同様に省略します。

## 6.7 補足: brmsを使わない分散分析モデルの推定
この節でデータの準備からモデル化,そして分析まで行います。

まずはダミー変数を作成するために、ユニーク値を確認します。

In [ ]:
print(df["weather"].unique().sort())

曇り(cloudy)を基準にして,晴れ(sunny)と雨(rainy)のダミー変数を作成します。

In [ ]:
df = (
    df
    .with_columns([
        pl.when( pl.col("weather") == "sunny" ).then(1)
        .otherwise(0)
        .alias("x1"),
        pl.when( pl.col("weather") == "rainy" ).then(1)
        .otherwise(0)
        .alias("x2"),
    ])
)
display(df.head())

numpyroに渡すためにjax.numpy配列に変換します。

In [ ]:
X1 = jnp.array(df["x1"], dtype = int)
X2 = jnp.array(df["x2"], dtype = int)
Y = jnp.array(df["sales"], dtype = float)

モデルを作成します。

In [ ]:
def model_dummy_variable(X1, X2, Y = None):
    '''
        第3部 6章 のビールの売り上げ
    '''
    # 説明変数をモデルに明示する
    # ベクトル化(学習用データを確率変数に割り当てるためのNumPyroのお作法)
    N = len(X1)
    with numpyro.plate("N", N):
        X1 = numpyro.deterministic("X1", X1)
        X2 = numpyro.deterministic("X2", X2)
    # 確率変数のパラメータの事前分布を設定する
    β0 = numpyro.sample("β0", dist.Normal(loc = 0, scale = 100))
    β1 = numpyro.sample("β1", dist.Normal(loc = 0, scale = 50))
    β2 = numpyro.sample("β2", dist.Normal(loc = 0, scale = 50))
    # リンク関数(恒等関数)を定義する
    μ = numpyro.deterministic("μ", β0 + β1 * X1 + β2 * X2)
    # 目的変数が従う正規分布の標準偏差の事前分布を設定する
    σ = numpyro.sample("σ", dist.HalfNormal(scale = 30))
    # 目的変数 y は説明変数 x の値に応じた平均値 μ をパラメータとする正規分布に従うと仮定します。
    with numpyro.plate("N", N):
        numpyro.sample("Y", dist.Normal(loc = μ, scale = σ), obs = Y)

モデルをプロットします。

In [ ]:
model_args = {
    "X1": X1,
    "X2": X2,
    "Y": Y
}
try_render_model(model_dummy_variable, render_name = "ビールの売り上げと天気の関係", **model_args)

ユーティリティ関数にモデルを渡してサンプリングします。

In [ ]:
model_args = {
    "X1": X1,
    "X2": X2,
    "Y": Y
}
mcmc = run_mcmc(
    model_dummy_variable,
    num_chains = 4,
    num_warmup = 1000,
    num_samples = 1000,
    thinning = 1,
    seed = 42,
    target_accept_prob = 0.9,
    **model_args
)
idata = az.from_numpyro(mcmc, log_likelihood = False)

サンプリングが上手くいったか確認します。

In [ ]:
az.plot_trace(idata, compact = False, var_names = ["β0", "β1", "β2", "σ"])
plt.tight_layout()

MCMCサンプルの概要を確認します。\
HDIを見ると $\beta 1$ は明確に正の値を取り、 $\beta 2$ は0をまたいでいます。

In [ ]:
az.summary(idata, var_names = ["β0", "β1", "β2"])

MCMCサンプルで天気別の売り上げの推定値を計算します。\
summary関数の通り晴れの日は売り上げが高くなるようです。

In [ ]:
# chain を統合します。
idata_stacked = idata["posterior"].stack(point = ["chain", "draw"])

# MCMCサンプルを抽出します。
β0 = idata_stacked["β0"].values
β1 = idata_stacked["β1"].values
β2 = idata_stacked["β2"].values

# 天気別のDtaFrameを作成して連結します。
df_cloudy = pl.DataFrame({
    "weather": ["cloudy"] * len(β0),
    "salse": (β0),
})
df_sunny = pl.DataFrame({
    "weather": ["sunny"] * len(β0),
    "salse": (β0 + β1),
})
df_rainy = pl.DataFrame({
    "weather": ["rainy"] * len(β0),
    "salse": (β0 + β1),
})
df_plot = pl.concat(
    items = [df_cloudy, df_sunny, df_rainy],
    how = "align",
)
del idata_stacked, β0, β1, β2, df_cloudy, df_sunny, df_rainy


ax = sns.barplot(
    data = df, x = "weather", y = "sales", hue = "weather",
    errorbar = "sd", capsize=.4,
    err_kws={"color": ".5", "linewidth": 2.5},
    linewidth = 0.1, edgecolor = ".1", facecolor=(0, 0, 0, 0),
)
ax.set_title("図3.6.2 推定されたビールの売り上げと天気の図示")

## 終わりに
